This tutorial introduces MongoEngine by means of example — we will walk through how to create a simple Tumblelog application. A tumblelog is a blog that supports mixed media content, including text, images, links, video, audio, etc. For simplicity’s sake, we’ll stick to text, image, and link entries. As the purpose of this tutorial is to introduce MongoEngine, we’ll focus on the data-modelling side of the application, leaving out a user interface.

Connecting to database

In [30]:
from mongoengine import *

connect('tumblelog')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In our Tumblelog application we need to store several different types of information. 
We will need to have a collection of users, so that we may link posts to an individual. 
We also need to store our different types of posts (eg: text, image and link) in the database. 
To aid navigation of our Tumblelog, posts may have tags associated with them, so that the list of posts shown to the user may be limited to posts that have been assigned a specific tag. Finally, it would be nice if comments could be added to posts. 
We’ll start with users, as the other document models are slightly more involved.

# Defining our documents

## Users

In [31]:

class User(Document):
    email = StringField(required=True)
    first_name = StringField(max_length=50)
    last_name = StringField(max_length=50)

## Post

We will store all of the posts in one collection and each post type will only store the fields it needs. 
If we later want to add video posts, we don’t have to modify the collection at all, we just start using the new fields we need to support video posts. 
This fits with the Object-Oriented principle of inheritance nicely. 
We can think of Post as a base class, and TextPost, ImagePost and LinkPost as subclasses of Post. In fact, MongoEngine supports this kind of modeling out of the box — all you need do is turn on inheritance by setting allow_inheritance to True in the meta:

In [32]:
class Post(Document):
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User)

    meta = {'allow_inheritance': True}

class TextPost(Post):
    content = StringField()

class ImagePost(Post):
    image_path = StringField()

class LinkPost(Post):
    link_url = StringField()

We are storing a reference to the author of the posts using a ReferenceField object. 
These are similar to foreign key fields in traditional ORMs, and are automatically translated into references when they are saved, and dereferenced when they are loaded.

## Tags

MongoDB allows us to store lists of items natively, so rather than having a link table, we can just store a list of tags in each post. 
So, for both efficiency and simplicity’s sake, we’ll store the tags as strings directly within the post, rather than storing references to tags in a separate collection. Especially as tags are generally very short (often even shorter than a document’s id), this denormalization won’t impact the size of the database very strongly

In [33]:
class Post(Document):
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User)
    tags = ListField(StringField(max_length=30))

The ListField object that is used to define a Post’s tags takes a field object as its first argument — this means that you can have lists of any type of field (including lists).

## Comments

Using MongoDB we can store the comments as a list of embedded documents directly on a post document. An embedded document should be treated no differently than a regular document; it just doesn’t have its own collection in the database. Using MongoEngine, we can define the structure of embedded documents, along with utility methods, in exactly the same way we do with regular documents:

In [34]:
class Comment(EmbeddedDocument):
    content = StringField()
    name = StringField(max_length=120)

We can then store a list of comment documents in our post document:

In [35]:
class Post(Document):
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User)
    tags = ListField(StringField(max_length=30))
    comments = ListField(EmbeddedDocumentField(Comment))

## Handling deletions of references

The ReferenceField object takes a keyword reverse_delete_rule for handling deletion rules if the reference is deleted. To delete all the posts if a user is deleted set the rule:

In [36]:
class Post(Document):
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User, reverse_delete_rule=CASCADE)
    tags = ListField(StringField(max_length=30))
    comments = ListField(EmbeddedDocumentField(Comment))

## Adding data to our Tumblelog

In [37]:
ross = User(email='ross@example.com', first_name='Ross', last_name='Lawley').save()

In [38]:
# or alternatively 
john = User(email='john@example.com')
john.first_name = 'John'
john.last_name = 'Smith'
john.save()

<User: User object>

In [39]:
## Add posts

In [40]:
post1 = TextPost(title='Fun with MongoEngine', author=john)
post1.content = 'Took a look at MongoEngine today, looks pretty cool.'
post1.tags = ['mongodb', 'mongoengine']
post1.save()

post2 = LinkPost(title='MongoEngine Documentation', author=ross)
post2.link_url = 'http://docs.mongoengine.com/'
post2.tags = ['mongoengine']
post2.save()

<LinkPost: LinkPost object>

## Accessing our data

In [41]:
for post in Post.objects:
    print(post.title)

Fun with MongoEngine
MongoEngine Documentation


RuntimeError: generator raised StopIteration

In [ ]:
for post in TextPost.objects:
    print(post.content)

In [ ]:
for post in Post.objects:
    print(post.title)
    print('=' * len(post.title))

    if isinstance(post, TextPost):
        print(post.content)

    if isinstance(post, LinkPost):
        print('Link: {}'.format(post.link_url))

In [ ]:
## Searching our posts by tag

In [ ]:
for post in Post.objects(tags='mongodb'):
    print(post.title)